# Spotify Streams in Europe & North America 2017-2021



<br><br>

## Index

- [Visualisation](#visualisation)
- [Data Preparation](#data-preparation)


<br><br><br>

<a name="visualisation"></a>



## Visualisation

Click on the bottom right of the visualisation to go full-screen. 

<br><br><br>

<div class='tableauPlaceholder' id='viz1644861167886' style='position: relative'><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SpotifyStreamsinEuropeNorthAmerica&#47;SpotifyStreamsEuropeandUS' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-GB' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1644861167886');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.minWidth='800px';vizElement.style.maxWidth='1700px';vizElement.style.width='100%';vizElement.style.minHeight='327px';vizElement.style.maxHeight='1027px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.minWidth='800px';vizElement.style.maxWidth='1700px';vizElement.style.width='100%';vizElement.style.minHeight='327px';vizElement.style.maxHeight='1027px';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='1877px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

<br><br><br>

<a name="data-preparation"></a>


## Data Preparation


The original Spotify Charts dataset has a size of over 3GB and would be very difficult to work with directly in Tableau, leading to a significant worse user experience. The rest of this notebook walks through how the dataset was prepared for the visualisation above. 

The Spotify dataset can be downloaded from Kaggle [here](https://www.kaggle.com/dhruvildave/spotify-charts), while the population dataset comes from Eurostat and can be found [here](https://ec.europa.eu/eurostat/web/population-demography/demography-population-stock-balance/database). 

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType, FloatType
from pyspark.sql.functions import col, date_format, from_unixtime, unix_timestamp

import os

In [32]:
# Input and output data dirs
raw_data = "raw_data/charts.csv"

# Initiate SparkContext and SparkSession
try:
    conf = SparkConf().setMaster("local[*]")
    sc = SparkContext(appName="Prepare Spotify Data")
    spark = SparkSession.builder.master("local").getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
except:
    print("SparkContext already running?")

spark

SparkContext already running?


Start by loading the data and specifying data type for each column. 

In [14]:
# Create schema to specify data type for each column
schema = StructType([
    StructField("Title", StringType(), True),
    StructField("Rank", IntegerType(), True),
    StructField("Date", DateType(), True),
    StructField("Artist", StringType(), True),
    StructField("Url", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("Chart", StringType(), True),
    StructField("Trend", StringType(), True),
    StructField("Streams", IntegerType(), True)
])


data = spark.read.csv(raw_data, 
                      header=True,
                      schema=schema)

data.show(5)
data.printSchema()

+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|               Title|Rank|      Date|              Artist|                 Url|   Region| Chart|        Trend|Streams|
+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|Chantaje (feat. M...|   1|2017-01-01|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|
|Vente Pa' Ca (fea...|   2|2017-01-01|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|
|Reggaetón Lento (...|   3|2017-01-01|                CNCO|https://open.spot...|Argentina|top200|    MOVE_DOWN| 210943|
|              Safari|   4|2017-01-01|J Balvin, Pharrel...|https://open.spot...|Argentina|top200|SAME_POSITION| 173865|
|         Shaky Shaky|   5|2017-01-01|        Daddy Yankee|https://open.spot...|Argentina|top200|      MOVE_UP| 153956|
+--------------------+----+----------+--

Create a new column containing year and month only and remove columns we don't need. 

In [15]:
# Create two new date columns for year and month
data = data.withColumn("YearMonth", date_format(data.Date, "yyyy-MM"))
data = data.withColumn("Year", date_format(data.Date, "yyyy"))

In [16]:
# Drop columns we're not interested in
for col in ['Date', 'Title', 'Rank', 'Artist', 'Url', 'Chart', 'Trend']:
    data = data.drop(col)

print(f"Number of rows in the dataset: {data.count()}\n")    
data.show(5)

Number of rows in the dataset: 26173514

+---------+-------+---------+----+
|   Region|Streams|YearMonth|Year|
+---------+-------+---------+----+
|Argentina| 253019|  2017-01|2017|
|Argentina| 223988|  2017-01|2017|
|Argentina| 210943|  2017-01|2017|
|Argentina| 173865|  2017-01|2017|
|Argentina| 153956|  2017-01|2017|
+---------+-------+---------+----+
only showing top 5 rows



In [17]:
# Display summary statistics
data.summary().show()

+-------+--------------------+-----------------+---------+------------------+
|summary|              Region|          Streams|YearMonth|              Year|
+-------+--------------------+-----------------+---------+------------------+
|  count|            26173514|         20318183| 26171372|          26171372|
|   mean|                null|55266.54339716303|     null|2019.1350362525893|
| stddev|                null|209602.0198801989|     null|  1.39037595219041|
|    min|               2nyce|             1001|  2017-01|              2017|
|    25%|                null|             3546|     null|            2018.0|
|    50%|                null|             9565|     null|            2019.0|
|    75%|                null|            35407|     null|            2020.0|
|    max|https://open.spot...|         19749704|  2021-12|              2021|
+-------+--------------------+-----------------+---------+------------------+



We're left with the columns we need and can make the following observations:

- There's a total of 26173514 rows in the dataset. Some of which contains Nulls. 
- Earliest date is January 2017 while most recent is December 2021. 
- The `Region` column seem to contain urls. Take a closer look at some of these next up.

In [18]:
# Display the first 10 rows containing a Url in the Region column
data.createOrReplaceTempView("data")
spark.sql("SELECT * FROM data WHERE Region LIKE '%https://open.spotify%'").show(10)

+--------------------+-------+---------+----+
|              Region|Streams|YearMonth|Year|
+--------------------+-------+---------+----+
|https://open.spot...|   null|  2017-02|2017|
|https://open.spot...|   null|  2017-02|2017|
|https://open.spot...|   null|  2017-02|2017|
|https://open.spot...|   null|  2017-02|2017|
|https://open.spot...|   null|  2017-08|2017|
|https://open.spot...|   null|  2017-08|2017|
|https://open.spot...|   null|  2017-08|2017|
|https://open.spot...|   null|  2017-02|2017|
|https://open.spot...|   null|  2017-02|2017|
|https://open.spot...|   null|  2017-08|2017|
+--------------------+-------+---------+----+
only showing top 10 rows



In [19]:
data.cube("Region").count().show(20, truncate=False)

+-----------------------------------------------------+------+
|Region                                               |count |
+-----------------------------------------------------+------+
|Chile                                                |448526|
|Hungary                                              |434873|
|https://open.spotify.com/track/72FJjSdfMCXiVcZTgR3Q6G|93    |
|Iceland                                              |302192|
|United Kingdom                                       |450606|
|Finland                                              |449042|
|Egypt                                                |180464|
|Greece                                               |370802|
|Denmark                                              |449295|
|France                                               |449265|
|Honduras                                             |376614|
|Austria                                              |454381|
|https://open.spotify.com/track/74ctCiq1T0Lm0HUt5YhZjg|

Unfortunately, there seem to be quite a lot of them. Fortunately, their total counts are very small in comparison and they all have `Null`streams. We can safely delete these rows without affecting the total streams. 

In [20]:
# Drop rows with Nulls
data = data.dropna(how="any")

In [21]:
# Check that there are no Nulls left
data.filter(" is NULL OR ".join(data.columns) + " is NULL").show()

+------+-------+---------+----+
|Region|Streams|YearMonth|Year|
+------+-------+---------+----+
+------+-------+---------+----+



In [22]:
data.summary().show()

+-------+---------+-----------------+---------+------------------+
|summary|   Region|          Streams|YearMonth|              Year|
+-------+---------+-----------------+---------+------------------+
|  count| 20318183|         20318183| 20318183|          20318183|
|   mean|     null|55266.54339716303|     null|2019.1431335174016|
| stddev|     null|209602.0198801989|     null|1.3881963686867085|
|    min|Argentina|             1001|  2017-01|              2017|
|    25%|     null|             3546|     null|            2018.0|
|    50%|     null|             9565|     null|            2019.0|
|    75%|     null|            35407|     null|            2020.0|
|    max|  Vietnam|         19749704|  2021-12|              2021|
+-------+---------+-----------------+---------+------------------+



We now have a total of around 20 million rows left. 

### Group data by Month and Region

Group by month and region to get the total streams per month and region. 

In [54]:
# Group by YearMonth and Region
yearMonth = data.groupBy(['Year','YearMonth', 'Region']) \
                .sum() \
                .withColumnRenamed("sum(Streams)", "StreamsPerMonth")

# Order by YearMonth and Region
yearMonth = yearMonth.orderBy(['YearMonth', 'Region'], ascending=[1, 1])

Confirm that the data looks nice and tidy. 

In [55]:
yearMonth.show(10)

+----+---------+---------+---------------+
|Year|YearMonth|   Region|StreamsPerMonth|
+----+---------+---------+---------------+
|2017|  2017-01|Argentina|      200416122|
|2017|  2017-01|Australia|      226426932|
|2017|  2017-01|  Austria|       24184172|
|2017|  2017-01|  Belgium|       37374447|
|2017|  2017-01|  Bolivia|        5514322|
|2017|  2017-01|   Brazil|      453286941|
|2017|  2017-01| Bulgaria|         589970|
|2017|  2017-01|   Canada|      216194987|
|2017|  2017-01|    Chile|      145855039|
|2017|  2017-01| Colombia|       60363348|
+----+---------+---------+---------------+
only showing top 10 rows



### Group by Year and Region


Group by year and region to get the total streams by year and region. 

In [62]:
# Group by Year and Region
year_totals = data.groupBy(['Year', 'Region']) \
                  .sum() \
                  .withColumnRenamed("sum(Streams)", "StreamsPerYear")

# Order by Year and Region
year_totals = year_totals.orderBy(['Year', 'Region'], ascending=[1, 1])

### Join above dataframes to get one complete dataset

Join the two dataframes and select the columns we want to keep; All columns from the `yearMonth` dataframe and the `StreamsPerYear` column from `year_totals`.

In [59]:
# Join the two dataframes and select columns to keep
final = yearMonth.join(year_totals,
                       (yearMonth.Year == year_totals.Year) &
                       (yearMonth.Region == year_totals.Region)) \
                 .select([yearMonth.Year, 
                          yearMonth.YearMonth,
                          yearMonth.Region,
                          yearMonth.StreamsPerMonth,
                          year_totals.StreamsPerYear])

final.show(10)

+----+---------+---------+---------------+--------------+
|Year|YearMonth|   Region|StreamsPerMonth|StreamsPerYear|
+----+---------+---------+---------------+--------------+
|2017|  2017-01|Argentina|      200416122|    2687519093|
|2017|  2017-02|Argentina|      193960016|    2687519093|
|2017|  2017-03|Argentina|      208207967|    2687519093|
|2017|  2017-08|Argentina|      233804765|    2687519093|
|2017|  2017-04|Argentina|      200942961|    2687519093|
|2017|  2017-05|Argentina|      201178865|    2687519093|
|2017|  2017-06|Argentina|      194446971|    2687519093|
|2017|  2017-07|Argentina|      221081939|    2687519093|
|2017|  2017-10|Argentina|      251138429|    2687519093|
|2017|  2017-09|Argentina|      238467300|    2687519093|
+----+---------+---------+---------------+--------------+
only showing top 10 rows



In [60]:
final.summary().show()

+-------+------------------+---------+---------+-------------------+--------------------+
|summary|              Year|YearMonth|   Region|    StreamsPerMonth|      StreamsPerYear|
+-------+------------------+---------+---------+-------------------+--------------------+
|  count|              3694|     3694|     3694|               3694|                3694|
|   mean|2019.0831077422847|     null|     null|3.039836877425555E8|3.6200024256819167E9|
| stddev|1.3968249481792545|     null|     null|9.741649787345997E8|1.164874034211760...|
|    min|              2017|  2017-01|Argentina|              75273|              279698|
|    25%|            2018.0|     null|     null|           20107937|           243142896|
|    50%|            2019.0|     null|     null|           60282461|           697970943|
|    75%|            2020.0|     null|     null|          261201576|          3241421071|
|    max|              2021|  2021-12|  Vietnam|         9476715327|         95741045288|
+-------+-

We're left with a total of 3694 rows from the original 26173514. Note that the `StreamPerYear` column displays the same value throughout all months of a year.


### Store to `.csv` file. 

In [61]:
# Save final data to .csv file
location = "cleaned_data/final_dataset"
final.write.csv(location, header=True)